In [4]:
import numpy as np
np.seterr(over='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'ignore', 'under': 'ignore'}

In [7]:
class neural_network():
    def __init__(self):
        np.random.seed(1)
        self.weights = []      # dict to hold weights
        self.num_layers = 1    # initial number of layers
        self.adjustments = []  # dict to hold adjustments
    
    def add_layer(self, shape):
        self.weights[self.num_layers] = np.vstack((2 * np.random.random(shape) - 1, 2 * np.random.random((1, shape[1])) - 1))
        self.adjustments[self.num_layers] = np.zeros(shape)
        self.num_layers += 1
        
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def predict(self, data):
        for layer in range(1, self.num_layers + 1):
            data = np.dot(data, self.weights[layer - 1][:, :-1]) + self.weights[layer - 1][:, -1] # + self.biases[layer]
            data = self.__sigmoid(data)
        return data
    
    def __forward_propagate(self, data):
        activation_values = {}
        activation_values[1] = data
        for layer in range(2, self.num_layer+1):
            data = np.dot(data.T, self.weights[layer - 1][:-1, :]) + self.weights[layer - 1][-1, :].T # + self.biases[layer]
            data = self.__sigmoid(data).T
            activation_values[layer] = data
        return activation_values
    
    def simple_error(self, outputs, targets):
        return targets - outputs
    
    def sum_squared_error(self, outputs, targets):
        return 0.5 * np.mean(np.sum(np.power(outputs - targets, 2), axis=1))
    
    def __back_propagate(self, output, target):
        deltas = {}
        deltas[self.num_layers] = output[self.num_layers] - target
        
        # delta of hidden layers
        for layer in reversed(range(2, self.num_layers)):
            a_val = output[layer]
            weights = self.weights[layer][:-1, :]
            prev_deltas = deltas[layer + 1]
            deltas[layer] = np.multiply(np.dot(weights, prev_deltas), self.__sigmoid_derivative(a_val))
            
        for layer in range(1, self.num_layers):
            self.adjustments[layer] += np.dot(deltas[layer + 1], output[layer].T).T
            
    def __gradient_descent(self, batch_size, learning_rate):
        for layer in range(1, self.num_layers):
            partial_d = (1 / batch_size) * self.adjustments[layer]
            self.weights[layer][:-1, :] += learning_rate * -partial_d
            self.weights[layer][-1, :] += learning_rate * 1e-3 * -partial_d[-1, :]
            
    def train(self, inputs, targets, num_epochs, learning_rate=1, stop_accuracy=1e-5):
        error = []
        for iteration in range(num_epochs):
            for i in range(len(inputs)):
                x = inputs[i]
                y = targets[i]
                output = self.__forward_propagate(x)
                
                loss = self.sum_squared_error(output[self.num_layers], y)
                error.append(loss)
                
                self.__back_propagate(output, y)
                
            self.__gradient_descent(i, learning_rate)
            
            if np.mean(error[-(i + 1):]) < stop_accuracy and iteration > 0:
                break
                
        return (np.asarray(error), iteration + 1)

In [8]:
if __name__ == '__main__':
    nn = neural_network()
    # add layers (input layer is created by default)
    nn.add_layer((2, 9))
    nn.add_layer((9, 1))
    
    # XOR function
    training_data = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]]).reshape(4, 2, 1)
    training_labels = np.asarray([[0], [1], [1], [0]])
    
    error, iteration = nn.train(training_data, training_labels, 5000)
    print('Error = ', np.mean(error[-4:]))
    print('Epochs needed to train = ', iteration)

IndexError: list assignment index out of range